## 実験

MASKの数、データ拡張の倍率、重要度算出手法を調整しながらデータ拡張する。



In [1]:
import os, subprocess

if 'COLAB_GPU' in os.environ:
    # Google Colabの場合
    from google.colab import drive
    drive.mount('/content/drive')
    base_path = "/content/drive/MyDrive/miti/ex/"

    #!pip install transformers sentencepiece datasets
    #上記はローカル環境でシンタックスエラーになる。代わりに以下のように実行。
    subprocess.run(["pip", "install", "torch","transformers", "fugashi", "unidic-lite"])
else:
    # ローカル環境の場合
    base_path = os.path.expanduser("./")

In [2]:
import os
import shutil
import random
import math
from collections import Counter
import numpy as np
import pandas as pd
import torch
import fugashi
import unidic_lite
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import transformers

ここでいろいろ設定することにする

In [3]:
#1つのデータでMASKに置き換える単語の数は？
n_masks = 3
#MASKのための重要度算出は？("tfidf" or "bm25")
mask_method = "tfidf"
#直後の相談者発話を入力データに含むか？
is_after_dialogue = True
#クライエント発話にもMASKを適用するか？
mask_client_dialogue = True
#データ拡張の倍率
multiplier_augmentation = 3

In [4]:
dataset_path = base_path + "datasets/original/dataset_original.csv"
data_path = base_path + "datasets/original/ver-20200312/"
#tfidf_path = "../tfidf/original_tfidf.txt"
model_name = "cl-tohoku/bert-base-japanese-v2"
gpt_path = base_path + "datasets/original/augmention_by_chatGPT.csv"

tokenizer = transformers.BertJapaneseTokenizer.from_pretrained(model_name)
gpt_df = pd.read_csv(gpt_path, encoding="utf-8")
data = {}
df = pd.DataFrame()
filelist = os.listdir(data_path)
for filename in filelist:
    if filename.split(".")[-1] != "csv":
        continue
    tmp_df = pd.read_csv(data_path + filename, encoding="cp932")
    tmp_dict = tmp_df.to_dict(orient="records")
    for line in gpt_df.iloc:
        before = line["before"]
        output = line["output"]
        after = line["after"] if type(line["after"]) != float else ""
        miticode = line["MITI-code"]
        source = line["source"]
        before_dict = {"NO":"Cl", "dialogue":before, "MITI_code":0, "target":""}
        output_dict = {"NO":"Co", "dialogue":output, "MITI_code":miticode, "target":""}
        after_dict = {"NO":"Cl", "dialogue":after, "MITI_code":0, "target":""}
        #print(filename, source)
        if source == filename:
            tmp_dict.append(before_dict)
            tmp_dict.append(output_dict)
            tmp_dict.append(after_dict)
    new_df = pd.DataFrame(tmp_dict)
    new_df.insert(4, "source",filename)
    df = pd.concat([df, new_df], axis=0)

In [5]:
df

,NO,dialogue,MITI_code,target,source
0,Co1,現状についてお話しいただきありがとうございました。そんな中、こちらのカウンセリングを受けて、...,3,転職に向けた行動を起こす,case1.csv
1,Cl1,そうですね。転職活動へのやる気を高めたいです。,7,NaN,case1.csv
2,Co2,やる気を高めて転職活動を行い、来年度からは新しい職場で働きたいわけですね。,2,NaN,case1.csv
3,Cl2,そうです。さすがに今の職場で来年度も働くイメージは持てません。,7,NaN,case1.csv
4,Co3,早急に脱出したいという思いが強いわけですね。,2,NaN,case1.csv
...,...,...,...,...,...
25,Cl13,そうですね。,0,NaN,case9.csv
26,Co14,もしかしたら、きちんと飲んでみると、効果も変わってくるんじゃないかと思うんですね。こちらで、...,4,NaN,case9.csv
27,Cl14,今日はちょっと難しいです。,8,NaN,case9.csv
28,Co15,そうですか。それでは、またお時間あるときにお立ち寄りください。,0,NaN,case9.csv


In [6]:
data_sep = []
for i, series in enumerate(df.iloc):
    if i == 0 or i == len(df)-1:
        continue
    if "Cl" not in df.iloc[i-1]["NO"]:
        continue
    elif "Co" not in df.iloc[i]["NO"]:
        continue
    source = series["source"]
    miti_code = series["MITI_code"]
    dialogue_before = df.iloc[i-1]["dialogue"] if "Cl" in df.iloc[i-1]["NO"] else ""
    dialogue = series["dialogue"]
    dialogue_after = df.iloc[i+1]["dialogue"] if "Cl" in df.iloc[i+1]["NO"] else ""
    if is_after_dialogue:
        sentence = dialogue_before + "[SEP]" + dialogue + "[SEP]" + dialogue_after
    else:
        sentence = dialogue_before + "[SEP]" + dialogue
    data_sep.append({"sentence":sentence, "miti_code":miti_code, "source":source})
df_sep = pd.DataFrame(data_sep)

In [7]:
for series in df_sep.iloc:
    print(series["sentence"])

そうですね。転職活動へのやる気を高めたいです。[SEP]やる気を高めて転職活動を行い、来年度からは新しい職場で働きたいわけですね。[SEP]そうです。さすがに今の職場で来年度も働くイメージは持てません。
そうです。さすがに今の職場で来年度も働くイメージは持てません。[SEP]早急に脱出したいという思いが強いわけですね。[SEP]はい。
はい。[SEP]それでもなかなかやる気が出てこないというのは、何度か失敗していたり、今の仕事が忙しかったりということがあるわけですね。[SEP]そうです。どうせこんな書類書いたって無駄だろうと思ってしまいます。
そうです。どうせこんな書類書いたって無駄だろうと思ってしまいます。[SEP]ただ、ご自身としては、そうも言っていられない状況だと。[SEP]そうですね。何とかして、脱出しなくてはいけないと思います。
そうですね。何とかして、脱出しなくてはいけないと思います。[SEP]来年度から別の職場で働くために出来る限りのことをしたいという思いはあるわけですね。[SEP]そうですね。現実的に、時間が無いということが大きいかもしれませんね。
そうですね。現実的に、時間が無いということが大きいかもしれませんね。[SEP]まずは、なんとか時間を確保する必要があると。[SEP]そうですね。やる気がないわけではないんだと思います。何とかして、転職先を見つけたいです。
そうですね。やる気がないわけではないんだと思います。何とかして、転職先を見つけたいです。[SEP]はい[SEP]ただ、時間的に厳しい中で、なかなかうまくいかず、落ち込んでいました。
ただ、時間的に厳しい中で、なかなかうまくいかず、落ち込んでいました。[SEP]忙しい中で、せっかく書いた書類がなかなか一次審査を通らないと、嫌になってしまうことはありますよね。ただ、実際は、今後も転職活動は続けていきたいということですね。[SEP]そうですね。一応会社の規定では、3か月前までに退職願を出さなくてはいけないことになっているのですが、当面はそれを目指して、それが難しくても、今後も転職活動を続けていく気ではいます。
そうですね。一応会社の規定では、3か月前までに退職願を出さなくてはいけないことになっているのですが、当面はそれを目指して、それが難しくても、今後も転職活動を続けていく気ではいます。[SE

まず、sep区切りしてから5分割して、5種類のデータセットを作成する。

train、valid、testがそれぞれ5種類できたら、trainだけ水増しして、3つセットでp1～p5/datasetに放り込む。

In [8]:
def cross_validation_split(data, num_split):
    data_shuffled = data.sample(frac=1).reset_index(drop=True)  # DataFrameをシャッフル

    split_size = len(data) // num_split  # DataFrameの長さを5で割る

    split_data = []
    for i in range(0, len(data), split_size):
        split_data.append(data_shuffled.iloc[i:i + split_size])  # ランダムに5等分してリストに追加

    return split_data[:num_split]

num_split = 5
split_df = cross_validation_split(df_sep, num_split)

In [9]:
for i in range(len(split_df)):
    print(len(split_df[i]))

44
44
44
44
44


交差検証用にデータセットを分割する。

In [10]:
train_df1 = pd.concat(split_df[0:3])
valid_df1 = split_df[3]
test_df1 = split_df[4]

train_df2 = pd.concat(split_df[1:4])
valid_df2 = split_df[4]
test_df2 = split_df[0]

train_df3 = pd.concat(split_df[2:])
valid_df3 = split_df[0]
test_df3 = split_df[1]

train_df4 = pd.concat([split_df[0], split_df[3], split_df[4]], ignore_index=True)
valid_df4 = split_df[1]
test_df4 = split_df[2]

train_df5 = pd.concat([split_df[0], split_df[1], split_df[4]], ignore_index=True)
valid_df5 = split_df[2]
test_df5 = split_df[3]

train_list = [train_df1, train_df2, train_df3, train_df4, train_df5]

これからtrain_listに各要素にMASKを施す。

その前に、全発話を含んだdfから単語リストを作成する。

In [11]:
i = 0
csvlist = []
for file in filelist:
    if file.split(".")[1] == "csv":
        csvlist.append(file)
print(csvlist)
def tokenize_with_subwords_and_pos(text, tokenizer):
    tagger = fugashi.Tagger()
    # fugashiを使って形態素解析
    tokens_with_pos = [(word.surface, word.feature.pos1) for word in tagger(text)]
    # WordPieceトークナイザーを使ってサブワード分割
    subword_tokens_with_pos = [[], []]
    for token, pos_info in tokens_with_pos:
        subwords = tokenizer.tokenize(token)
        # サブワードごとにトークンと品詞情報のペアを作成
        for subword in subwords:
            subword_tokens_with_pos[0].append(subword)
            subword_tokens_with_pos[1].append(pos_info)
    return subword_tokens_with_pos

def tokenize_with_subwords_and_pos_v2(text, tokenizer):
    tagger = fugashi.Tagger()
    # fugashiを使って形態素解析
    tokens_with_pos = [(word.surface, word.feature.pos1) for word in tagger(text)]
    # WordPieceトークナイザーを使ってサブワード分割
    subword_tokens_with_pos = [[], []]
    for token, pos_info in tokens_with_pos:
        #トークンをさらにサブワードに分解
        subwords = tokenizer.tokenize(token)
        is_subword = False
        #サブワードに品詞をつける
        for subword in subwords:
            if "#" in subword:
                is_subword = True
        if is_subword:
            for subword in subwords:
                subword_tokens_with_pos[0].append(subword)
                subword_tokens_with_pos[1].append(pos_info)
        else:
            subword_tokens_with_pos[0].append(token)
            subword_tokens_with_pos[1].append(pos_info)
    return subword_tokens_with_pos


['case1.csv', 'case2!.csv', 'case3.csv', 'case4.csv', 'case5!.csv', 'case6!.csv', 'case7.csv', 'case8!.csv', 'case9.csv']


In [12]:
word_list_from_tokenizer = {}
word_list_from_fugashi = {}
for key in csvlist:
    print(key)
    tmp_df = df[df["source"] == key]
    dialogues = tmp_df["dialogue"].str.strip("\u3000").tolist()
    sum_dialogues = "".join(dialogues)
    tokenized_dialogues = tokenize_with_subwords_and_pos_v2(sum_dialogues, tokenizer)
    word_list_from_fugashi[key] = tokenized_dialogues
    tokenized_dialogues_by_tokenizer = tokenizer.tokenize(sum_dialogues)
    word_list_from_tokenizer[key] = tokenized_dialogues_by_tokenizer

case1.csv
case2!.csv
case3.csv
case4.csv
case5!.csv
case6!.csv
case7.csv
case8!.csv
case9.csv


In [13]:
conversion_dict = {
    "たって": ["たっ", "##て"],
    "良さ": ["良", "##さ"],
    "取れれ": ["取れ", "##れ"],
    "なんか": ["なん", "##か"],
    "わかん": ["わか", "##ん"],
    "エイデンヘ": ["エイ", "##デン", "##ヘ"],
    "時々":["時", "々"],
    "よさ": ["よ", "##さ"],
    "借りれ": ["借り", "##れ"],
    "借りれる": ["借り", "##れる"],
    "ガッツン": ["ガッツ", "##ン"],
    "，":[","],
    "（":["("],
    "）":[")"],
    "１":["1"],
    "２":["2"],
    "１０":["10"],
    "3１":["31"],
    "３":["3"],
    "５":["5"],
    "？":["?"],
    "ＭＩ":["MI"],
    "㎏":["kg"],
    "…": [".", ".", "."],
    "！":["!"],
    "％":["%"],
    "｡":["。"],
    "－":["-"],
    "．":["."],
    "～":["~"]
}

In [14]:
new_word_list = {}
for key in csvlist:
    new_word_list[key] = [[], []]
    word_and_pos_in_file = word_list_from_fugashi[key]
    tmp_word_list = []
    tmp_pos_list = []
    for i in range(len(word_and_pos_in_file[0])):
        token = word_and_pos_in_file[0][i]
        pos = word_and_pos_in_file[1][i]
        if token in conversion_dict.keys():
            tokens = conversion_dict[token]
            for token in tokens:
                tmp_word_list.append(token)
                tmp_pos_list.append(pos)
        else:
            tmp_word_list.append(token)
            tmp_pos_list.append(pos)
    new_word_list[key] = [tmp_word_list, tmp_pos_list]

In [15]:
i = 0
csvlist = []
for file in filelist:
    if file.split(".")[1] == "csv":
        csvlist.append(file)
print(csvlist)
def tokenize_with_subwords_and_pos(text, tokenizer):
    tagger = fugashi.Tagger()
    # fugashiを使って形態素解析
    tokens_with_pos = [(word.surface, word.feature.pos1) for word in tagger(text)]
    # WordPieceトークナイザーを使ってサブワード分割
    subword_tokens_with_pos = [[], []]
    for token, pos_info in tokens_with_pos:
        subwords = tokenizer.tokenize(token)
        # サブワードごとにトークンと品詞情報のペアを作成
        for subword in subwords:
            subword_tokens_with_pos[0].append(subword)
            subword_tokens_with_pos[1].append(pos_info)
    return subword_tokens_with_pos

def tokenize_with_subwords_and_pos_v2(text, tokenizer):
    tagger = fugashi.Tagger()
    # fugashiを使って形態素解析
    tokens_with_pos = [(word.surface, word.feature.pos1) for word in tagger(text)]
    # WordPieceトークナイザーを使ってサブワード分割
    subword_tokens_with_pos = [[], []]
    for token, pos_info in tokens_with_pos:
        #トークンをさらにサブワードに分解
        subwords = tokenizer.tokenize(token)
        is_subword = False
        #サブワードに品詞をつける
        for subword in subwords:
            if "#" in subword:
                is_subword = True
        if is_subword:
            for subword in subwords:
                subword_tokens_with_pos[0].append(subword)
                subword_tokens_with_pos[1].append(pos_info)
        else:
            subword_tokens_with_pos[0].append(token)
            subword_tokens_with_pos[1].append(pos_info)
    return subword_tokens_with_pos


['case1.csv', 'case2!.csv', 'case3.csv', 'case4.csv', 'case5!.csv', 'case6!.csv', 'case7.csv', 'case8!.csv', 'case9.csv']


In [16]:
word_list_from_tokenizer = {}
word_list_from_fugashi = {}
for key in csvlist:
    print(key)
    tmp_df = df[df["source"] == key]
    dialogues = tmp_df["dialogue"].str.strip("\u3000").tolist()
    sum_dialogues = "".join(dialogues)
    tokenized_dialogues = tokenize_with_subwords_and_pos_v2(sum_dialogues, tokenizer)
    word_list_from_fugashi[key] = tokenized_dialogues
    tokenized_dialogues_by_tokenizer = tokenizer.tokenize(sum_dialogues)
    word_list_from_tokenizer[key] = tokenized_dialogues_by_tokenizer

case1.csv
case2!.csv
case3.csv
case4.csv
case5!.csv
case6!.csv
case7.csv
case8!.csv
case9.csv


In [17]:
new_word_list = {}
for key in csvlist:
    new_word_list[key] = [[], []]
    word_and_pos_in_file = word_list_from_fugashi[key]
    tmp_word_list = []
    tmp_pos_list = []
    for i in range(len(word_and_pos_in_file[0])):
        token = word_and_pos_in_file[0][i]
        pos = word_and_pos_in_file[1][i]
        if token in conversion_dict.keys():
            tokens = conversion_dict[token]
            for token in tokens:
                tmp_word_list.append(token)
                tmp_pos_list.append(pos)
        else:
            tmp_word_list.append(token)
            tmp_pos_list.append(pos)
    new_word_list[key] = [tmp_word_list, tmp_pos_list]

In [44]:
def compute_bm25(documents, query, k1=2.0, b=0.75, csvlist=['case1.csv', 'case2!.csv', 'case3.csv', 'case4.csv', 'case5!.csv', 'case6!.csv', 'case7.csv', 'case8!.csv', 'case9.csv']):
    word_counts = {}  # 単語のカウントを格納する空の辞書
    # 各文書を処理して単語をカウントし、辞書に格納する
    for i, doc in enumerate(documents, 0):
        # 辞書に単語をカウントして格納
        word_count = {}
        for word in doc:
            if word not in word_count:
                word_count[word] = 1
            else:
                word_count[word] += 1

        # 各文書の単語カウントをword_countsに追加
        word_counts[csvlist[i]] = word_count
    # DataFrameに単語カウントを格納
    df = pd.DataFrame(word_counts).fillna(0).astype(int)
    print(df)
    # 各文書の単語数を取得
    doc_lengths = df.sum(axis=0)
    #print(doc_lengths)
    # 全文書の平均単語数を計算
    avg_doc_length = doc_lengths.mean()
    #print(avg_doc_length)
    score_for_query = {}
    for word in query:
        print("computing...",word)
        scores = []
        word_series = df.loc[word]
        # IDF (Inverse Document Frequency)
        idf = math.log((len(df.columns) - word_series[word_series != 0].count() + 0.5) / (word_series[word_series != 0].count() + 0.5))
        #print(word, ":idf,",idf)
        for document in df.columns:
            doc = df.loc[:][document]
            doc_counter = dict(doc[doc != 0])  # 0でない値のみ取得して辞書に変換
            #print(doc_counter)
            # 各文書の単語数を取得
            doc_length = doc_lengths[document]
            # 各クエリの単語に対するBM25スコアを計算
            # TF (Term Frequency)
            tf = doc_counter.get(word, 0) / doc.sum()
            #print(document,word,":tf,",tf)
            # bm25 score
            if tf == 0:
                score = -1
            else:
                score = idf * ((tf * (k1 + 1)) / (tf + k1 * (1 - b + b * doc_length / avg_doc_length)))

            scores.append(score)
        score_for_query[word] = scores
    return score_for_query

# 例
documents = [
    "私は研究をしています。",
    "明日、ゼミで研究の進捗を発表します。",
    "明日はチキン弁当弁当を食べます。",
    "今日は研究と研究と研究をします。"*2
]

query = ["研究", "弁当", "明日", "を", "。", "今日"]

bm25_score = compute_bm25([tokenizer.tokenize(doc) for doc in documents], query)
for score in bm25_score:
    print(score, bm25_score[score], sep="\t")

     case1.csv  case2!.csv  case3.csv  case4.csv
私            1           0          0          0
は            1           0          1          2
研究           1           1          0          6
を            1           1          1          2
し            1           1          0          2
て            1           0          0          0
い            1           0          0          0
ます           1           1          1          2
。            1           1          1          2
明日           0           1          1          0
、            0           1          0          0
ゼミ           0           1          0          0
で            0           1          0          0
の            0           1          0          0
進捗           0           1          0          0
発表           0           1          0          0
チキン          0           0          1          0
弁当           0           0          2          0
食べ           0           0          1          0
今日           0      

In [19]:
#filelist
count = 0
incorrect_list = []
for key in csvlist:
    j = 0
    tokens_from_tokenizer = word_list_from_tokenizer[key]
    tokens_from_fugashi = new_word_list[key]
    print(key)
    for i in range(len(tokens_from_tokenizer)):    
        correct_token = tokens_from_tokenizer[i]
        my_token = tokens_from_fugashi[0][j]
        my_token_pos_info = tokens_from_fugashi[1][j]
        is_correct = correct_token == my_token

        if not is_correct:
            k = j + 1 if j+1 < len(tokens_from_fugashi[0]) else j
            l = j + 2 if j+2 < len(tokens_from_fugashi[0]) else j
            m = j + 3 if j+3 < len(tokens_from_fugashi[0]) else j
            n = j + 4 if j+4 < len(tokens_from_fugashi[0]) else j
            if tokens_from_fugashi[0][k] == correct_token:
                j = k
                #print(correct_token, tokens_from_fugashi[0][k], "True",  sep="\t")
                count -= 1
            elif tokens_from_fugashi[0][l] == correct_token:
                j = l
                #print(correct_token, tokens_from_fugashi[0][l], "True", sep="\t")
            elif tokens_from_fugashi[0][m] == correct_token:
                j = m
                #print(correct_token, tokens_from_fugashi[0][m], "True", sep="\t")

            else:
                incorrect_list.append((key, j, correct_token, my_token_pos_info))
                j -= 1
                print(correct_token, my_token, is_correct, sep="\t")
                print("前のトークン:", tokens_from_fugashi[0][j])
                count += 1


        else:
            #print(correct_token , my_token, is_correct, my_token_pos_info, sep="\t")
            a = 0
        j += 1


case1.csv
case2!.csv
case3.csv
case4.csv
##と	と	False
前のトークン: ##え
##と	と	False
前のトークン: ##え
な	なん	False
前のトークン: ,
ん	なん	False
前のトークン: ,
##,	,	False
前のトークン: 々
case5!.csv
case6!.csv
case7.csv
case8!.csv
case9.csv
効	効か	False
前のトークン: が
##か	効か	False
前のトークン: が


In [20]:
print(incorrect_list)

for incorrect in incorrect_list:
    key = incorrect[0]
    index = incorrect[1]
    correct = incorrect[2]
    pos = incorrect[3]
    if correct != "ん" and correct != "##,":
        new_word_list[key][0][index] = correct
    elif correct == "##,":
        new_word_list[key][0][index+1] = correct 
    else:
        new_word_list[key][0].insert(index+1, correct)
        new_word_list[key][1].insert(index+1, pos)

[('case4.csv', 95, '##と', '助詞'), ('case4.csv', 486, '##と', '助詞'), ('case4.csv', 497, 'な', '代名詞'), ('case4.csv', 497, 'ん', '代名詞'), ('case4.csv', 1310, '##,', '補助記号'), ('case9.csv', 223, '効', '動詞'), ('case9.csv', 223, '##か', '動詞')]


tokenizerによる分割と完全一致するか確認する。

In [21]:
for key in csvlist:
    print(key)
    print(new_word_list[key][0] == word_list_from_tokenizer[key])

case1.csv
True
case2!.csv
True
case3.csv
True
case4.csv
True
case5!.csv
True
case6!.csv
True
case7.csv
True
case8!.csv
True
case9.csv
False


In [22]:
docs = []
for key in csvlist:
    print(key)
    docs.append(new_word_list[key][0])

case1.csv
case2!.csv
case3.csv
case4.csv
case5!.csv
case6!.csv
case7.csv
case8!.csv
case9.csv


In [23]:
words_set = set()
stop_words = []
for i, doc in enumerate(docs):
    for word in doc:
        if word not in stop_words:
            words_set.add(word)
print("単語の種類数：",len(words_set))
#print(words_set)

単語の種類数： 1378


bm25による重要度算出の準備終わり。

ここからTF-IDFの方を算出する。

TF-IDFの場合は名詞と形容詞のみ重要度を算出するため、単語種類数がbm25の方と一致しない

In [24]:
words_extracted_by_pos = []
for key in csvlist:
    tmp_words = []
    print(key)
    words = new_word_list[key][0]
    pos_list = new_word_list[key][1]
    for i in range(len(words)):
        word = words[i]
        pos = pos_list[i]
        if pos=="名詞" or pos == "形容詞":
            tmp_words.append(word)
    words_extracted_by_pos.append(" ".join(tmp_words))

case1.csv
case2!.csv
case3.csv
case4.csv
case5!.csv
case6!.csv
case7.csv
case8!.csv
case9.csv


In [25]:
vectorizer = TfidfVectorizer(use_idf=True, stop_words=[])
tfidf = vectorizer.fit_transform(words_extracted_by_pos)

In [26]:
terms = vectorizer.get_feature_names_out()
print(terms.shape)

(517,)


bm25の重要度算出

In [27]:
scores = compute_bm25(docs, words_set)

computing... しまっ
computing... ##切れ
computing... 惣
computing... ##る
computing... 負わ
computing... 使い分け
computing... 貸し
computing... 仕事
computing... A
computing... 減
computing... 高め
computing... 行動
computing... 性
computing... ニコ
computing... 挑戦
computing... 代用
computing... 会話
computing... ワイン
computing... 担当
computing... て
computing... 負担
computing... 起き
computing... やっ
computing... エイ
computing... ##せ
computing... ##ぐ
computing... つき
computing... だけ
computing... なに
computing... させ
computing... 大分
computing... 入社
computing... でき
computing... ##こ
computing... 者
computing... 年
computing... 前向き
computing... じゃ
computing... 様子
computing... ##送
computing... ちゃ
computing... ##刻
computing... 豊富
computing... ##たい
computing... 調整
computing... たくさん
computing... 退職
computing... 心配
computing... ##える
computing... ガム
computing... こう
computing... ##ゃく
computing... 額
computing... ##起
computing... ##上げ
computing... 「
computing... 起こし
computing... ##らっ
computing... こんな
computing... ##れる
computing... 昔
comp

computing... ひと
computing... すべて
computing... 増加
computing... ##らせ
computing... 使え
computing... 使用
computing... しっかり
computing... なお
computing... 審査
computing... 誘っ
computing... 近所
computing... 小
computing... 職場
computing... 現実
computing... ながら
computing... 変わっ
computing... 私
computing... 興味
computing... 通じ
computing... O
computing... ##y
computing... 主
computing... ごく
computing... 低下
computing... 具体
computing... あと
computing... 望ま
computing... 行う
computing... 6
computing... 変わら
computing... 飲み
computing... とり
computing... 点数
computing... ローン
computing... 対し
computing... 小さく
computing... マックス
computing... 分から
computing... きれ
computing... ##らしい
computing... いずれ
computing... 比べる
computing... ##ン
computing... 有
computing... 行い
computing... なれる
computing... 家
computing... 置い
computing... わけ
computing... 明確
computing... 月
computing... そろ
computing... 最
computing... 連絡
computing... ど
computing... 必要
computing... 特定
computing... 薬
computing... 開発
computing... 1
computing... いや
computing... だ


computing... いっ
computing... 水
computing... 修正
computing... 現状
computing... チュ
computing... 自信
computing... ##ん
computing... ##む
computing... 嬉
computing... 実践
computing... たく
computing... ##チン
computing... 原因
computing... 君
computing... 実質
computing... 余
computing... 弁当
computing... 方法
computing... 失敗
computing... 返済
computing... 法
computing... 寝
computing... 初め
computing... にくい
computing... どんどん
computing... 悩ん
computing... 今
computing... しまう
computing... マイナス
computing... 一
computing... おい
computing... 役立つ
computing... 除い
computing... 貢献
computing... 言っ
computing... ブラック
computing... スク
computing... 申し込み
computing... 達成
computing... ねぇ
computing... カル
computing... 200
computing... 参加
computing... まし
computing... 含ん
computing... 作業
computing... 対応
computing... 任せ
computing... 変わる
computing... は
computing... まっ
computing... 事
computing... 関係
computing... おっ
computing... 出
computing... 話し
computing... 思い
computing... 自分
computing... ガッツ
computing... ##り
computing... 行き
computing... 減らす

In [28]:
scores_df = pd.DataFrame.from_dict(scores, orient="index", columns=csvlist)

In [29]:
def extract_largest_values(lst, num_values):
    # 数値のみを含むリストを作成し、それぞれの値とインデックスを保持するタプルのリストを生成します
    numeric_list = [(val, idx) for idx, val in enumerate(lst) if isinstance(val, (int, float))]

    # 数値の大きい順にソートします
    sorted_numeric_list = sorted(numeric_list, key=lambda x: x[0], reverse=True)

    # 指定された数だけ大きい値とそのインデックスを抽出します
    largest_values = sorted_numeric_list[:num_values]

    return largest_values

# テスト用のリスト
numbers_list = [10, 5, 8, 20, 3, 12, 9]

# 例として、3つの最大値を取得します
result = extract_largest_values(numbers_list, 3)
print("取り出した最大の数値とそのインデックス:", result)

取り出した最大の数値とそのインデックス: [(20, 3), (12, 5), (10, 0)]


In [30]:
def mask_func(text, scores, file_name, csvlist, mask_num):
    scorelist = []
    for i, f in enumerate(csvlist):
        if f == file_name:
            file_number = i
            break
        file_number = None
    #print(file_number)
    maskword_indices = []
    masked_indices = []
    tokenized_text = tokenizer.tokenize(text)
    if mask_num > len(tokenized_text):
        mask_num = len(tokenized_text)
    for i, token in enumerate(tokenized_text):
        #print(token, end="\t")
        if token not in scores.keys():
            #print("ないよ")
            scorelist.append(-1)
            continue
        else:
            scorelist.append(scores[token][file_number])
    maskids = [item[1] for item in extract_largest_values(scorelist, mask_num)]
    masktokens = [tokenized_text[i] for i in maskids]
    label = [-100]*len(tokenized_text)
    #print(mask_num)
    #print(masktokens)
    for i in range(mask_num):
        masktoken = masktokens[i]
        maskid = maskids[i]
        maskword_id = tokenizer.encode(masktoken, add_special_tokens=False)[0]
        maskword_indices.append(maskword_id)
        tokenized_text[maskid] = "[MASK]"
    masked_text = "".join(tokenized_text).replace("#", "")
    assert all([tokenized_text[idx]==4] for idx in maskids)
    print(text,"->",masked_text)
    return masked_text, maskword_indices


mask_func("勉強をもっとしたいという気持ちがある。",scores, "case2!.csv", csvlist, 4)

勉強をもっとしたいという気持ちがある。 -> [MASK]を[MASK]し[MASK]という[MASK]がある。


('[MASK]を[MASK]し[MASK]という[MASK]がある。', [15945, 13512, 15460, 11722])

In [31]:
def mask_func_tfidf(text, tfidf, terms,  file_name, csvlist, mask_num):
    for i, f in enumerate(csvlist):
        if f == file_name:
            file_number = i
            break
        file_number = None
    tokenized_text = tokenizer.tokenize(text)
    scorelist = []
    for i, token in enumerate(tokenized_text):
        scorelist.append(-1)
        for j, term in enumerate(terms):
            if token == term:
                #print(term)
                #print(tfidf.toarray()[file_number, j])
                scorelist[i] = tfidf.toarray()[file_number, j]
    #print(scorelist)
    maskids = [item[1] for item in extract_largest_values(scorelist, mask_num) if item[0]!=-1]
    #print(maskids)
    maskedtokenids = [tokenizer.encode(tokenized_text[i], add_special_tokens=False)[0] for i in maskids]
    #print(maskedtokenids)
    for i in maskids:
        tokenized_text[i] = "[MASK]"
    if len(maskids) == 0:
        tokenized_text[0] = "[MASK]"
    masked_text = "".join(tokenized_text).replace("#", "")
    print(text,"->",masked_text)
    return masked_text, maskedtokenids

mask_func_tfidf("勉強をもっとしたいという気持ちがある。", tfidf, terms, "case2!.csv", csvlist, 2)

勉強をもっとしたいという気持ちがある。 -> [MASK]をもっとしたいという[MASK]がある。


('[MASK]をもっとしたいという[MASK]がある。', [15945, 15460])

In [32]:
print(scores["、"])

[-0.12144062136481962, -0.24214020632882047, -0.07429747669112191, -0.013897728400770812, -0.6728758992545683, -0.19059887962221822, -0.18396445346293955, -0.17968048895936042, -0.30966092120163996]


In [33]:
print(mask_func_tfidf("やる気を高めて転職活動を行い、来年度からは新しい職場で働きたいわけですね。", tfidf, terms, "case1.csv", csvlist, 4))

やる気を高めて転職活動を行い、来年度からは新しい職場で働きたいわけですね。 -> [MASK]気を高めて[MASK][MASK]を行い、来[MASK]からは新しい職場で働きたいわけですね。
('[MASK]気を高めて[MASK][MASK]を行い、来[MASK]からは新しい職場で働きたいわけですね。', [26493, 11286, 16595, 11637])


In [34]:
def create_masked_dataset(train_list, mask_num, method, mask_client_dialogue, scores=scores, tfidf=tfidf, terms=terms):
    max_token_length = 300
    masked_train_list = {}

    for i, train in enumerate(train_list):
        key_name = "train" + str(i)
        masked_train_list[key_name] = []
        for series in train.iloc:
            source = series["source"]
            miti_code = series["miti_code"]
            if mask_client_dialogue:
                dialogue = series["sentence"]
            else:
                dialogue_cl1, dialogue = series["sentence"].split("[SEP]")

            if method == "tfidf":
                masked_dialogue, maskword_ids = mask_func_tfidf(dialogue, tfidf, terms, source, csvlist, mask_num=mask_num)
            elif method == "bm25":
                masked_dialogue, maskword_ids = mask_func(dialogue, scores, source, csvlist, mask_num=mask_num) 
            if mask_client_dialogue:
                masked_sentence = masked_dialogue
            else:             
                masked_sentence = dialogue_cl1 + "[SEP]" + masked_dialogue
            #print(masked_sentence)
            ids_for_assertion = tokenizer(masked_sentence)["input_ids"]
            masked_ids = [i for i in range(len(ids_for_assertion)) if ids_for_assertion[i] == 4]
            if len(masked_ids) >= 2:
                #assert all([label==-100 if idx!=masked_ids[0] or idx!=masked_ids[1] else label!=-100 for idx, label in enumerate(labels)])
                assert all([ids_for_assertion[idx] == 4 for idx in masked_ids])
            else:
                #assert all([label==-100 if idx!=masked_ids[0] else label!=-100 for idx, label in enumerate(labels)])
                assert ids_for_assertion[masked_ids[0]]==4

            masked_train_list[key_name].append({"sentence":series["sentence"],"masked_sentence":masked_sentence, "labels":[], "miti_code":miti_code,  "maskword_id":maskword_ids, "masked_ids":masked_ids, "source":source})
    return masked_train_list
    
masked_train_list = create_masked_dataset(train_list, n_masks, mask_method, mask_client_dialogue)

わ，わかりません。私が言いたいのは，息子の周りでは吸わないように気をつけているということです。車の中では吸いませんし，息子が家にいるときは外に吸いに行きます。とにかく私にたばこが悪いということは分かっているし，息子にとっても悪いことだと分かっているからタバコの周りにいないでほしいと言っているし，本当に気をつけています。[SEP]エイデンのためにもあなたの健康のためにもタバコをやめたほうがいいと思います。子どもの周りでタバコを吸うことが影響するのは耳の感染症だけではないのは知っていましたか？[SEP]うん -> わ,わかりません。私が言いたいのは,息子の周りでは吸わないように気をつけているという[MASK]です。車の中では吸いませんし,息子が家にいるときは外に吸いに行きます。とにかく私にたばこが悪いという[MASK]は分かっているし,息子にとっても悪い[MASK]だと分かっているからタバコの周りにいないでほしいと言っているし,本当に気をつけています。[SEP]エイデンのためにもあなたの健康のためにもタバコをやめたほうがいいと思います。子どもの周りでタバコを吸うことが影響するのは耳の感染症だけではないのは知っていましたか?[SEP]うん
わかりました。とにかくわかりました。[SEP]息子さんが何度も中耳炎を繰り返していることは、あなたにとっても息子さんにとっても大きなストレスですね。タバコの煙が子どもの耳感染のリスクファクターの一つであることを考慮すると、あなたが喫煙していることが関連している可能性があります。息子さんの周りで吸わないように気をつけているというあなたの努力は理解していますが、それでもタバコの影響は考慮に入れなければならないかもしれません。[SEP] -> わかりました。とにかくわかりました。[SEP]息子さんが何度も中耳炎を繰り返している[MASK]は、あなたにとっても息子さんにとっても大きなストレスですね。タバコの煙が子どもの耳感染のリスクファクターの一つである[MASK]を考慮すると、あなたが喫煙している[MASK]が関連している可能性があります。息子さんの周りで吸わないように気をつけているというあなたの努力は理解していますが、それでもタバコの影響は考慮に入れなければならないかもしれません。[SEP]
ええ，その通りです。[SEP]以前やめたときはど

わかりません，私…助けがほしい，どういった助けを必要としているかわからない。[SEP]わかりました，もし何か興味があれば，もちろんお話することができます。実際にいろんな方の禁煙を成功させている新たな選択肢もたくさんあります。あなたが試せる薬物療法も…[SEP]薬は好きではありません。 -> わかりません,私...助けがほしい,どういった助けを必要としているかわから[MASK]。[SEP]わかりました,もし何か興味があれば,もちろんお話する[MASK]ができます。実際にいろんな方の[MASK]を成功させている新たな選択肢もたくさんあります。あなたが試せる薬物療法も...[SEP]薬は好きではありません。
そうですね。目覚ましをたくさんかけたり、夜は早めに負担に入るようにしたり...[SEP]そうですね、夜早く寝る努力や目覚ましをかける工夫、いいですね。ちょっとずつでも、そういう努力が結果につながると思いますよ。[SEP]そうですね。ただ、難しいですよね。 -> そうですね。[MASK]ましをたくさんかけたり、夜は早めに[MASK]に入るようにしたり...[SEP]そうですね、夜早く寝る努力や[MASK]ましをかける工夫、いいですね。ちょっとずつでも、そういう努力が結果につながると思いますよ。[SEP]そうですね。ただ、難しいですよね。
わからない。[SEP]勉強を続けたいという思いはあるものの、会社での使用機会がないため、やらなくてもいいのではという気持ちがあるということですね。役に立たないかもしれないという思いもある一方で、やった方がいいと感じる気持ちが少し大きいということですね。楽しく勉強できる環境があれば、もっと前向きになれるかもしれません。[SEP] -> わからない。[SEP][MASK]を続けたいという思いはあるものの、[MASK]での使用機会がないため、やらなくてもいいのではという気持ちがあるということですね。役に立たないかもしれないという思いもある一方で、やった方がいいと感じる気持ちが少し大きいということですね。楽しく[MASK]できる環境があれば、もっと前向きになれるかもしれません。[SEP]
とても嬉しいです[SEP]OK，ではもしよければ，あなたが来られるときに一度予約を取り直して，そのことについて話をしましょう。[SEP]そうしたいと思います，

ええ[SEP]では，エイデンヘ抗生物質の処方菱を書きました。申し上げにくいのですが，彼のカルテを見て，ここ最近頻繁に耳の感染症を起こしているのが少し気になります。また，家庭内でどなたか喫煙されているという項目にチェックをされていますね。このことについてもう少しお聞かせ願えますか。[SEP]ええと，それって私と子どもの問題でしよ。私は煙草を吸ってますよ。う－ん，息子のそばでは吸わないように本当に気をつけています。でも妊娠していた時は除いて10年吸い続けています｡シングルマザーでいることはとてもストレスフルでまた，仕事もあります。だから，また吸いはじめてしまいました。 -> ええ[SEP]では,エイデンヘ抗生物質の処方菱を書きました。申し上げにくいのですが,彼のカルテを見て,ここ最近頻繁に耳の感染症を起こしているのが少し気になります。また,家庭内でどなたか喫煙されているという項目にチェックをされていますね。この[MASK]についてもう少しお聞かせ願えますか。[SEP]ええと,それって私と子どもの問題でしよ。私は煙草を吸ってますよ。う-ん,息子のそばでは吸わ[MASK]ように本当に気をつけています。でも妊娠していた時は除いて10年吸い続けています。シングルマザーでいる[MASK]はとてもストレスフルでまた,仕事もあります。だから,また吸いはじめてしまいました。
もう、私は吸っていないって・・・いえ，私，私，私やめようとしてきたけれど，でもすごく難しくてどうやっていいか分からないんです。[SEP]今がやめるときです。吸い続けることはできません。彼のためだけでなくあなたのためにも言います。肺がんや肺気腫，口腔がん，心疾患などのりスクも増えるんですよ。[SEP]わかっています。わかっています。聞いたことがあるし，以前人に言われたこともあります。全部聞きました。ただどうやっていいかわからないんです。どうやって禁煙したらいいの？すごく難しい。 -> もう、私は吸っていないって・・・いえ,私,私,私やめようとしてきたけれど,でもすごく難しくてどうやっていいか分からないんです。[SEP]今がやめるときです。吸い続ける[MASK]はできません。彼のためだけでなくあなたのためにも言います。肺がんや肺気腫,口腔がん,心疾患などのりスクも増えるんですよ。[SEP]わかっています。わかっています

うれしい。そうなんや～。これや～！っという。おいんしんや～！！って思ったけど、飲む・食うしちゃうと、大昔のあの時代に戻るのかな？なんて。[SEP]怖いとうい感覚と、でも反対の我慢できそうという感覚もあるんじゃなかと。　[SEP]うーん。そうそう、今のご飯の食べ方やったら、いけるんやろな～っと思うけど（笑）。 -> うれしい。そうなんや~。これや~!っという。おいんしんや~!!って思ったけど、飲む・食うしちゃうと、大昔のあの[MASK]に戻るのかな?なんて。[SEP][MASK]とうい感覚と、でも反対の我慢できそうという感覚もあるんじゃなかと。[SEP]うーん。そうそう、今の[MASK]の食べ方やったら、いけるんやろな~っと思うけど(笑)。
実際違う仕事が入ってくると、また、そうしようかなぁみたいな。[SEP]じゃあ、仕事の忙しさとかと、英語についてもう少し勉強したら、海外の情報を得たり学会に出たり、なにか広がりそうだなぁっていうところで、なんか悩まれてるっていうところ。[SEP]んー、そうね、悩んでる…まぁちょっとやって、オンライン英会話とかトライして。ただ忙しい時は難しい、そんなところを繰り返してる状況ですかね。 -> 実際違う仕事が入ってくると、また、そうしようかなぁみたいな。[SEP]じゃあ、仕事の忙しさとかと、[MASK]についてもう少し[MASK]したら、海外の情報を得たり[MASK]に出たり、なにか広がりそうだなぁっていうところで、なんか悩まれてるっていうところ。[SEP]んー、そうね、悩んでる...まぁちょっとやって、オンライン英会話とかトライして。ただ忙しい時は難しい、そんなところを繰り返してる状況ですかね。
そうなんですよ...逆に起きられない時は、寝るのが遅いのかもしれません。[SEP]今年度も保健指導にご参加くださりありがとうございます。では早速ですが、健診結果の振返り、食事・運動・その他で主にタバコについて確認させてください。どうでしょうか？[SEP]5年前に比べると、良くなっている。特に体重が80㎏だったから～。 -> そうなんですよ...逆に起きられない時は、寝るのが遅いのかもしれません。[SEP]今年度も保健指導にご参加くださりありがとうございます。では早速ですが、健診結果の振返り、[MASK]・運動・その他で主に[MASK]について確

はい。いずれも帰宅してしまうと、なかなか作業の時間が取れないことが多いので、帰宅前にやれるといいですかね。例えば、水、金や月、火で日中に1時間取れなかったときには、帰宅途中でカフェやレストランによって1時間作業をして帰るとか。[SEP]いいですね！そのような工夫をすると、1日1時間転職のための活動をすることに対する自信度は、10点満点でどれくらいになりますか？[SEP]8，9割出来るのではないでしょうか。100％というのは難しいと思ので、これくらいなら十分かと思います。 -> はい。いずれも帰宅してしまうと、なかなか作業の[MASK]が取れないことが多いので、帰宅前にやれるといいですかね。例えば、水、金や月、火で日中に1[MASK]取れなかったときには、帰宅途中でカフェやレストランによって1[MASK]作業をして帰るとか。[SEP]いいですね!そのような工夫をすると、1日1時間転職のための活動をすることに対する自信度は、10点満点でどれくらいになりますか?[SEP]8,9割出来るのではないでしょうか。100%というのは難しいと思ので、これくらいなら十分かと思います。
そうなんですよね。[SEP]でもそれがなかなか難しいと。[SEP]はい。気をつけてはいるんですが… -> そうなんですよね。[SEP]でもそれがなかなか[MASK]と。[SEP]はい。気をつけてはいるんですが...
えぇ[SEP]やりたいという気持ちがあると最初におっしゃっていたのですが、やったらどんないいことがあると思いますか[SEP]出来てるかわからないけど、自分が成長してる気がするし。する。自分のことも知れる。かな。 -> えぇ[SEP]やりたいという[MASK]があると最初におっしゃっていたのですが、やったらどんないい[MASK]があると思いますか[SEP]出来てるかわからないけど、自分が成長してる気がするし。する。自分の[MASK]も知れる。かな。
そうですね。ただ、難しいですよね。[SEP]そうだね。でも、来られているときもあるよね。その時はどんな感じかな？[SEP]うぅん．．．たまたま起きるとか、そんなに意識していなくても起きられるんですよね。 -> そうですね。ただ、[MASK]ですよね。[SEP]そうだね。でも、来られているときもあるよね。その時はどんな[MASK]かな?[SEP]うぅ

...

そうですね。目覚ましをたくさんかけたり、夜は早めに負担に入るようにしたり...[SEP]早寝早起きを心がけることは、健康にも良い影響をもたらします。そのような生活習慣を実践しようとするあなたの姿勢は、とても素晴らしいです。[SEP]そうですね。ただ、難しいですよね -> そうですね。[MASK]ましをたくさんかけたり、夜は早めに[MASK]に入るようにしたり...[SEP]早寝早起きを心がける[MASK]は、健康にも良い影響をもたらします。そのような生活習慣を実践しようとするあなたの姿勢は、とても素晴らしいです。[SEP]そうですね。ただ、難しいですよね
そうです，ええ。彼は端息はないし，彼の友達が抱えているようないろいろな問題もないし。それに，ずっと前，やめようと思ったのですが，今はどうやったらできるのかわかりません，わからないのです。[SEP]妊娠した時はどうしてやめようと思ったのですか？[SEP]ええと，彼がお腹にいるときは，なんでも分け合っているわけだから，胎児に影響があるかもと分かっていたし，息子に何かがあったら生きていけないと思っていました。 -> そうです,ええ。彼は端息は[MASK]し,彼の友達が抱えているようないろいろな問題も[MASK]し。それに,ずっと前,やめようと思ったのですが,今はどうやったらできるのかわかりません,わから[MASK]のです。[SEP]妊娠した時はどうしてやめようと思ったのですか?[SEP]ええと,彼がお腹にいるときは,なんでも分け合っているわけだから,胎児に影響があるかもと分かっていたし,息子に何かがあったら生きていけないと思っていました。
わからない。[SEP]前はカウンセリングの勉強が役に立ち、会社の人と一緒にすることが楽しかったですが、現在は環境が変わり、勉強することへのモチベーションが低下しているようです。それでも、以前の経験を思い出して再度挑戦しようと思うことがあり、今日もこうして参加しているということですね。[SEP] -> わからない。[SEP]前はカウンセリングの[MASK]が役に立ち、[MASK]の人と一緒にすることが楽しかったですが、現在は環境が変わり、[MASK]することへのモチベーションが低下しているようです。それでも、以前の経験を思い出して再度挑戦しようと思うことがあり、今日もこうして参加している

そうですね。やっぱり、自分のために時間は使うべきだという思いがあります。今の仕事は、それが嫌いなわけではないですが、自分が今やるべきことはそれではないと思います。自分の人生やキャリアの優先順位を上げるべきだと思います。[SEP]仕事は仕事で大事だけれど、今自分のために何が必要かということを考えると、それは、転職に向けた活動を優先することだということですね。[SEP]はい。 -> そうですね。やっぱり、自分のために[MASK]は使うべきだという思いがあります。今の仕事は、それが嫌いなわけではないですが、自分が今やるべきことはそれではないと思います。自分の人生やキャリアの優先順位を上げるべきだと思います。[SEP]仕事は仕事で大事だけれど、今自分のために何が必要かということを考えると、それは、[MASK]に向けた[MASK]を優先することだということですね。[SEP]はい。
うぅん．．．たまたま起きるとか、そんなに意識していなくても起きられるんですよね。[SEP]来られるときは、意識せずにも起きられるのか！それはいいね！[SEP]そうなんですよ…逆に起きられない時は、寝るのが遅いのかもしれません。 -> うぅん...たまたま起きるとか、そんなに[MASK]していなくても起きられるんですよね。[SEP]来られるときは、[MASK]せずにも起きられるのか!それはいいね![SEP]そうなんですよ...逆に起きられない時は、寝るのが[MASK]のかもしれません。
はい。せっかくなので、大胆に時間を取ってみようと思います。本当に、来年度も今の職場で働き続けるというのは、自分にとって良いことではないと思うので。[SEP]時間管理に関して少し困っているようですね。特に大きな問題があったわけではないけれど、朝の時間に間に合わせることが難しかったということが理解できます。日常生活での小さな挑戦にも前向きに取り組んでいることは、とても良いことだと思います。[SEP]実際どうなるかはわからないですけどね。計画的に進められるといいとは思います。 -> はい。せっかくなので、大胆に[MASK]を取ってみようと思います。本当に、来年度も今の職場で働き続けるというのは、自分にとって良いことではないと思うので。[SEP][MASK]管理に関して少し困っているようですね。特に大きな問題があったわけではな

そうですね。それだけのせいではないですが、仕事を持ち帰っていることも多いですかね。[SEP]一つは、仕事を持ち帰えっているせいもあると。それはご苦労様。申し訳なかったね。[SEP]いえいえ。たまたま期限のある仕事だったので。 -> そうですね。それだけのせいではないですが、[MASK]を持ち帰っている[MASK]も多いですかね。[SEP]一つは、仕事を[MASK]えっているせいもあると。それはご苦労様。申し訳なかったね。[SEP]いえいえ。たまたま期限のある仕事だったので。
はい。[SEP]あなたは、1日1時間の時間を転職活動に費やすことを意識しており、帰宅途中にカフェやレストランを利用することで、この目標を8、9割達成できると考えています。このような具体的な計画は、あなたの転職活動への自信を高めるのに役立つでしょう。[SEP]はい。 -> はい。[SEP]あなたは、1日1[MASK]の[MASK]を[MASK]活動に費やすことを意識しており、帰宅途中にカフェやレストランを利用することで、この目標を8、9割達成できると考えています。このような具体的な計画は、あなたの転職活動への自信を高めるのに役立つでしょう。[SEP]はい。
エアコン買ったり支払いにまた考えたいなと思ってるんですけど。[SEP]支払いが終わった後の事とかは、どーしよかなぁとか？[SEP]もう一回お金の使い分けを、また父母と一緒にできるんであれば考えていきたいなと思ってます。 -> [MASK]コン買ったり[MASK]にまた考えたいなと思ってるんですけど。[SEP][MASK]が終わった後の事とかは、どーしよかなぁとか?[SEP]もう一回お金の使い分けを、また父母と一緒にできるんであれば考えていきたいなと思ってます。
ええ，そうすべきだと分かっています，毎年新年に“今年は禁煙する”といいますが，何かあるとできない，やめられないんです。何もなければやめられるのに・・・。[SEP]それはあなたの「やるべき仕事」の中で，最優先ではないのね[SEP]ええ -> ええ,そうすべきだと分かっています,毎年新年に“今年は禁煙する”といいますが,何かあるとでき[MASK],やめられ[MASK]んです。何もなければやめられるのに・・・。[SEP]それはあなたの「やるべき仕事」の中で,最優先では[MASK]のね[SEP]

実際どうなるかはわからないですけどね。計画的に進められるといいとは思います。[SEP]1日1時間、転職活動のための時間に使うということの重要度について点数を付けてもらいたいのですが、とても重要だというのを10点として、あまり重要でないというのを0点とすると、Aさんにとっては、何点くらいになりそうですか？[SEP]重要度でいうと、8とか9とか。10までいかないにしても、かなり高いと思います。 -> 実際どうなるかはわからないですけどね。計画的に進められるといいとは思います。[SEP]1日1[MASK]、[MASK]活動のための[MASK]に使うということの重要度について点数を付けてもらいたいのですが、とても重要だというのを10点として、あまり重要でないというのを0点とすると、Aさんにとっては、何点くらいになりそうですか?[SEP]重要度でいうと、8とか9とか。10までいかないにしても、かなり高いと思います。
そりゃそうですね。[SEP]ですよね。医師はそのことについて何と言っているんですか？[SEP]何と言っているわけでもないですけど、薬が変わることもあれば、もう少し様子を見ようとなることもありますね。 -> そりゃそうですね。[SEP]ですよね。医師はその[MASK]について何と言っているんですか?[SEP]何と言っているわけでもないですけど、薬が変わる[MASK]もあれば、もう少し様子を見ようとなる[MASK]もありますね。
そうですね。やる気というよりも、淡々と、計画的に、応募書類を出し続けるということですかね。気分的なことよりも、現職との両立がキーになりそうです。[SEP]やる気というよりも、いかに現在のお仕事と両立させながら、応募を継続していくかが重要だろうということですね。[SEP]はい -> そうですね。やる気というよりも、淡々と、計画的に、応募書類を出し続けるという[MASK]ですかね。気分的な[MASK]よりも、現職との両立がキーになりそうです。[SEP]やる気というよりも、いかに現在のお仕事と両立させながら、応募を継続していくかが重要だろうという[MASK]ですね。[SEP]はい
そうなんですよ…逆に起きられない時は、寝るのが遅いのかもしれません。[SEP]遅く寝ているせいで、朝定時に起きられない。[SEP]そうですね。 -> そうなんですよ...

はい。[SEP]あなたは、月曜日と火曜日に予期せぬ業務が発生したり、水曜日と金曜日に疲れて帰宅することが、転職活動における時間確保の障壁となっていると感じていますね。このような状況に対処する方法を見つけることが、あなたの自信度を向上させる鍵になりそうです。[SEP]はい。 -> はい。[SEP]あなたは、月曜日と火曜日に予期せぬ業務が発生したり、水曜日と金曜日に疲れて帰宅することが、[MASK][MASK]における[MASK]確保の障壁となっていると感じていますね。このような状況に対処する方法を見つけることが、あなたの自信度を向上させる鍵になりそうです。[SEP]はい。
わかりません。[SEP]あなたが私に禁煙しますといってくれることをまってます。ホントに大事なことです。[SEP]そうね，それを全部見てみます。何かできるようなことを見つけて頑張ってみます。それで私の主治医に話をしてみます。 -> わかりません。[SEP]あなたが私に禁煙しますといってくれる[MASK]をまってます。ホントに大事な[MASK]です。[SEP]そうね,それを全部見てみます。何かできるような[MASK]を見つけて頑張ってみます。それで私の主治医に話をしてみます。
そういうわけじゃにんですけど、あきらめてますよね。悪くならないだけいいかって。[SEP]確かにそうですね。悪くなっていないのは良いことですね。もっと効く方法があったら、なお良いという感じですかね。[SEP]そりゃそうですね。 -> そういうわけじゃにんですけど、あきらめてますよね。悪くなら[MASK]だけいいかって。[SEP]確かにそうですね。悪くなってい[MASK]のは良い[MASK]ですね。もっと効く方法があったら、なお良いという感じですかね。[SEP]そりゃそうですね。
うん、知りたい。[SEP]自分が成長している気持ちで勉強できたらいいなと。[SEP]うん。 -> うん、知りたい。[SEP]自分が成長している[MASK]で[MASK]できたら[MASK]なと。[SEP]うん。
いいえ。大丈夫です。[SEP]わかりました。それでは、今日はこれで面談を終えたいと思います。またいつでもご連絡ください。本日はありがとうございました。[SEP]ありがとうございました。 -> [MASK]え。大丈夫です。[SEP]わかりました。それで

In [35]:
n_masks

3

入力用のデータができた。

後はmask推定をしてもらい、上位2件のデータを水増しデータセットの中に入れる

今回は5種類の水増しデータセットを作成するため、元データと水増しデータの対応をしっかり行う

In [36]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(device)

cuda


In [37]:
model = transformers.BertForMaskedLM.from_pretrained(model_name)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-v2 were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


MASKが二つある場合も出力はおそらく同じ。

BertのOutput.logitsには全トークン分の行列が含まれているため、そこからMASKした2つののインデックスを取り出せばいいと思われる

でもテストしておく

In [38]:
print(masked_train_list)

{'train0': [{'sentence': 'わ，わかりません。私が言いたいのは，息子の周りでは吸わないように気をつけているということです。車の中では吸いませんし，息子が家にいるときは外に吸いに行きます。とにかく私にたばこが悪いということは分かっているし，息子にとっても悪いことだと分かっているからタバコの周りにいないでほしいと言っているし，本当に気をつけています。[SEP]エイデンのためにもあなたの健康のためにもタバコをやめたほうがいいと思います。子どもの周りでタバコを吸うことが影響するのは耳の感染症だけではないのは知っていましたか？[SEP]うん', 'masked_sentence': 'わ,わかりません。私が言いたいのは,息子の周りでは吸わないように気をつけているという[MASK]です。車の中では吸いませんし,息子が家にいるときは外に吸いに行きます。とにかく私にたばこが悪いという[MASK]は分かっているし,息子にとっても悪い[MASK]だと分かっているからタバコの周りにいないでほしいと言っているし,本当に気をつけています。[SEP]エイデンのためにもあなたの健康のためにもタバコをやめたほうがいいと思います。子どもの周りでタバコを吸うことが影響するのは耳の感染症だけではないのは知っていましたか?[SEP]うん', 'labels': [], 'miti_code': 4, 'maskword_id': [11143, 11143, 11143], 'masked_ids': [31, 66, 79], 'source': 'case3.csv'}, {'sentence': 'わかりました。とにかくわかりました。[SEP]息子さんが何度も中耳炎を繰り返していることは、あなたにとっても息子さんにとっても大きなストレスですね。タバコの煙が子どもの耳感染のリスクファクターの一つであることを考慮すると、あなたが喫煙していることが関連している可能性があります。息子さんの周りで吸わないように気をつけているというあなたの努力は理解していますが、それでもタバコの影響は考慮に入れなければならないかもしれません。[SEP]', 'masked_sentence': 'わかりました。とにかくわかりました。[SEP]息子さんが何度も中耳炎を繰り返している[MASK]は、あなたに

In [39]:
model.to(device)
data_augmented = {}
model.eval()
sp_ids_list = [tokenizer.cls_token_id, tokenizer.pad_token_id]
for key in masked_train_list.keys():
    train_data = masked_train_list[key]
    data_augmented[key] = []
    for data in train_data:
        count = 0
        sentence = data["sentence"]
        masked_sentence = data["masked_sentence"]
        miti_code = data["miti_code"]
        source = data["source"]
        maskword = data["maskword_id"]
        masked_ids = data["masked_ids"]
        inputs = tokenizer(masked_sentence)
        with torch.no_grad():
            outputs = model(torch.tensor([inputs["input_ids"]]).to(device))
        predictions = [outputs.logits[0, mid].topk(multiplier_augmentation+1).indices for mid in masked_ids]
        #print(predictions)
        #すべてのMASKを埋めることができる1組の予測を取り出す
        for i in range(len(predictions[0])):
            if count >= multiplier_augmentation:
                break
            prediction_list = []
            new_sentence_ids = inputs["input_ids"]
            preds = [predictions[j][i] for j in range(len(predictions))]
            #print(preds)
            #各予測をMASK位置に埋め込んでいく
            for k in range(len(preds)):
                mid = masked_ids[k]
                p = preds[k]
                prediction_list.append(p.item())
                decodeword = tokenizer.decode(p)
                new_sentence_ids[mid] = p
            new_sentence_ids = [index for index in new_sentence_ids if index not in sp_ids_list][:-1]
            decoded_sentence = tokenizer.decode(new_sentence_ids)
            new_sentence = "".join(decoded_sentence.split(" "))
            print(new_sentence)
            if maskword != prediction_list:
                count += 1
                data_augmented[key].append({"sentence":new_sentence, "miti_code":data["miti_code"], "augmented":True, "source":data["source"]})
        data_augmented[key].append({"sentence":sentence,"miti_code":data["miti_code"], "augmented":False, "source":data["source"]})




わ,わかりません。私が言いたいのは,息子の周りでは吸わないように気をつけているということです。車の中では吸いませんし,息子が家にいるときは外に吸いに行きます。とにかく私にたばこが悪いということは分かっているし,息子にとっても悪いことだと分かっているからタバコの周りにいないでほしいと言っているし,本当に気をつけています。[SEP]エイデンのためにもあなたの健康のためにもタバコをやめたほうがいいと思います。子どもの周りでタバコを吸うことが影響するのは耳の感染症だけではないのは知っていましたか?[SEP]うん
わ,わかりません。私が言いたいのは,息子の周りでは吸わないように気をつけているという事です。車の中では吸いませんし,息子が家にいるときは外に吸いに行きます。とにかく私にたばこが悪いというのは分かっているし,息子にとっても悪いタバコだと分かっているからタバコの周りにいないでほしいと言っているし,本当に気をつけています。[SEP]エイデンのためにもあなたの健康のためにもタバコをやめたほうがいいと思います。子どもの周りでタバコを吸うことが影響するのは耳の感染症だけではないのは知っていましたか?[SEP]うん
わ,わかりません。私が言いたいのは,息子の周りでは吸わないように気をつけているというのです。車の中では吸いませんし,息子が家にいるときは外に吸いに行きます。とにかく私にたばこが悪いという事は分かっているし,息子にとっても悪いものだと分かっているからタバコの周りにいないでほしいと言っているし,本当に気をつけています。[SEP]エイデンのためにもあなたの健康のためにもタバコをやめたほうがいいと思います。子どもの周りでタバコを吸うことが影響するのは耳の感染症だけではないのは知っていましたか?[SEP]うん
わ,わかりません。私が言いたいのは,息子の周りでは吸わないように気をつけているというものです。車の中では吸いませんし,息子が家にいるときは外に吸いに行きます。とにかく私にたばこが悪いという気持ちは分かっているし,息子にとっても悪いのだと分かっているからタバコの周りにいないでほしいと言っているし,本当に気をつけています。[SEP]エイデンのためにもあなたの健康のためにもタバコをやめたほうがいいと思います。子どもの周りでタバコを吸うことが影響するのは耳の感染症だけではないのは

まぁ、やってみて、、、これで1回やってみて、うん、減るなら減るで。[SEP]それが大きくなるだけでも、それでもええかなっと![SEP]ようは、いかに自分で自制するっていうのか~。
まぁ、やってみて、、、これで1回やってみて、うん、減るなら減るで。[SEP]気分が少なくなるだけでも、それでもええかなっと![SEP]ようは、いかに自分で自制するっていうことか~。
まぁ、やってみて、、、これで1回やってみて、うん、減るなら減るで。[SEP]気持ちが強くなるだけでも、それでもええかなっと![SEP]ようは、いかに自分で自制するっていうものか~。
そうすると、やっぱどうか。(少し間を置いて)一番怖いのはどうして太るのが怖いかなっと。[SEP]仕事が増えちゃうこと。[SEP]やめていた間に、仕出し弁当が、ガッツンときてもうて。メシうめーっていうのがあったから。
そうすると、やっぱ何か。(少し間を置いて)一番怖いのはダイエットして太るのが怖いかなっと。[SEP]体重が増えちゃうこと。[SEP]やめていた間に、仕出し弁当が、ガッツンときてもうて。メシうめーっていうのがあったから。
そうすると、やっぱなぜか。(少し間を置いて)一番怖いのはこうして太るのが怖いかなっと。[SEP]ストレスが増えちゃうこと。[SEP]やめていた間に、仕出し弁当が、ガッツンときてもうて。メシうめーっていうのがあったから。
そうですね...毎日30分位が現実的かという思いと、1時間取れれば、だいぶ前進しそうだという思いがあります。[SEP]1時間は、少し、現実的ではないかもしれない。一方で、1年ぐらい取れた方が、転職活動は進むだろうということですね。[SEP]そうなんですよね...
そうですね...毎日30分位が現実的かという思いと、1年取れれば、だいぶ前進しそうだという思いがあります。[SEP]1年は、少し、現実的ではないかもしれない。一方で、1時間ぐらい取れた方が、転職活動は進むだろうということですね。[SEP]そうなんですよね...
そうですね...毎日30分位が現実的かという思いと、1日取れれば、だいぶ前進しそうだという思いがあります。[SEP]1年間は、少し、現実的ではないかもしれない。一方で、1日ぐらい取れた方が、転職活動は進むだろうということですね。[SEP]そうなんですよね...
はい。気をつ

うん。[SEP]今のスポーツとして続けられそうだなというのは10点だとしたら何点ぐらい?[SEP]3
うん。[SEP]今の人生として続けられそうだなということは10点満点としたら何点ぐらい?[SEP]3
うん。[SEP]今の職業として続けられそうだなという気持ちは10点あるとしたら何点ぐらい?[SEP]3
はい。[SEP]なんだか飲みたいんですね。せっかく飲んでるのに...[SEP]はぁ...
はい。[SEP]なんだかやりたいのですね。せっかく飲んでるのに...[SEP]はぁ...
はい。[SEP]なんだかしたいもんですね。せっかく飲んでるのに...[SEP]はぁ...
うん、少しあるかな。[SEP]今後この会を通じて信頼関係が出来てみんなとの話ができたらもしかしたら、参加できるかもしれない。[SEP]難しい。わかんない。
うん、少しあるかな。[SEP]今後この会を通じて人間関係が出来てみんなとも話ができたらもしかしたら必ず参加できるかもしれない。[SEP]難しい。わかんない。
うん、少しあるかな。[SEP]今後この会を通じて交友関係が出来てみんなとお話ができたらもしかしたら全員参加できるかもしれない。[SEP]難しい。わかんない。
3[SEP]1とか2じゃないのはどういうところから、今3になってるんですか?[SEP]今日もこういうのがあるよと誘ってくれた人がいて、遊びに来てくれた人がいるから一人じゃないから。
3[SEP]1とか2じゃいうのはどういうところから、今3になってるんですか?[SEP]今日もこういうのがあるよと誘ってくれた人がいて、一緒に来てくれた人がいるから一人じゃねえから。
3[SEP]1とか2じゃってのはどういうところから、今3になってるんですか?[SEP]今日もこういうのがあるよと誘ってくれた人がいて、迎えに来てくれた人がいるから一人じゃいいから。
はい。[SEP]それでもなかなかその気が出てこないというのは、何度か失敗していたり、今の仕事が忙しかったりということがあるわけですね。[SEP]そうです。どうせこんなに書いたって無駄だろうと思ってしまいます。
はい。[SEP]それでもなかなかいい気が出てこないというのは、何度か失敗していたり、今の仕事が忙しかったりというのがあるわけですね。[SEP]そうです。どうせこんなこと書いたって無駄だろうと思っ

そりゃそうですね。[SEP]ですよね。医師はその薬について何と言っているんですか?[SEP]何と言っているわけでもないですけど、薬が変わることもあれば、もう少し様子を見ようとなることもありますね。
そりゃそうですね。[SEP]ですよね。医師はそのことについて何と言っているんですか?[SEP]何と言っているわけでもないですけど、薬が変わる場合もあれば、もう少し様子を見ようとなる場合もありますね。
そりゃそうですね。[SEP]ですよね。医師はその点について何と言っているんですか?[SEP]何と言っているわけでもないですけど、薬が変わる時もあれば、もう少し様子を見ようとなるときもありますね。
そうですね。やる気というよりも、淡々と、計画的に、応募書類を出し続けるということですかね。気分的なことよりも、現職との両立がキーになりそうです。[SEP]やる気というよりも、いかに現在のお仕事と両立させながら、応募を継続していくかが重要だろうということですね。[SEP]はい
そうですね。やる気というよりも、淡々と、計画的に、応募書類を出し続けるというんですかね。気分的な仕事よりも、現職との両立がキーになりそうです。[SEP]やる気というよりも、いかに現在のお仕事と両立させながら、応募を継続していくかが重要だろうという感じですね。[SEP]はい
そうですね。やる気というよりも、淡々と、計画的に、応募書類を出し続けるというものですかね。気分的な面よりも、現職との両立がキーになりそうです。[SEP]やる気というよりも、いかに現在のお仕事と両立させながら、応募を継続していくかが重要だろうというんですね。[SEP]はい
そうですね。やる気というよりも、淡々と、計画的に、応募書類を出し続けるという感じですかね。気分的なものよりも、現職との両立がキーになりそうです。[SEP]やる気というよりも、いかに現在のお仕事と両立させながら、応募を継続していくかが重要だろうというのですね。[SEP]はい
そうなんですよ...逆に起きられない時は、寝るのがいいのかもしれません。[SEP]毎日寝ているので、朝定時に起きられない。[SEP]そうですね。
そうなんですよ...逆に起きられない時は、寝るのが怖いのかもしれません。[SEP]朝寝ているだけで、朝定時に起きられない。[SEP]そうですね。
そうなんですよ..

はい。[SEP]あなたは、帰宅途中にカフェやレストランで作業時間を確保することを考えているようですね。このような工夫をすることで、転職活動に必要な1日1時間の時間を実質的に確保し、目標達成に向けた自信が高まると感じているのですね。[SEP]はい。
はい。[SEP]あなたは、帰宅途中にカフェやレストランで作業場所を確保することを考えているようですね。このような工夫をすることで、転職活動に必要な1日1食の睡眠を実質的に確保し、目標達成に向けた自信が高まると感じているのですね。[SEP]はい。
はい。[SEP]あなたは、帰宅途中にカフェやレストランで作業スペースを確保することを考えているようですね。このような工夫をすることで、転職活動に必要な1日1回の仕事を実質的に確保し、目標達成に向けた自信が高まると感じているのですね。[SEP]はい。
はい。[SEP]あなたは、帰宅途中にカフェやレストランで作業量を確保することを考えているようですね。このような工夫をすることで、転職活動に必要な1日1本のスペースを実質的に確保し、目標達成に向けた自信が高まると感じているのですね。[SEP]はい。
ええ,私たちにとってすごくストレスなんです。でもすごく悩んでいて。[SEP]子どもに多発する耳感染の主なリスクファクターのひとつには,タバコの煙にさらされることがあげられます。タバコを吸いますか?[SEP]ええ,吸います。でも彼の周りで吸いません。彼の周りでは吸わないように気をつけています。
ええ,私たちにとってすごくストレスなんです。いつもすごく悩んでいて。[SEP]子どもに多発する耳感染の主なリスクファクターのひとつには,たばこの煙にさらされる事があげられます。タバコを吸いますか?[SEP]ええ,吸います。でも彼の周りで吸いません。彼の周りでは吸わないように気をつけています。
ええ,私たちにとってすごくストレスなんです。私もすごく悩んでいて。[SEP]子どもに多発する耳感染の主なリスクファクターのひとつには,煙草の煙にさらされるリスクがあげられます。タバコを吸いますか?[SEP]ええ,吸います。でも彼の周りで吸いません。彼の周りでは吸わないように気をつけています。
そう!やめている間に、そこの仕出し弁当は元々不味かったのに、その弁当でさえ、中身が何でもうめって思った。[SEP]美味しさをす

...

そうですね。昼ましをたくさんかけたり、夜は早めに風呂に入るようにしたり...[SEP]色々工夫してくれてるみたいだね。ありがとう。それが効果を発揮してくれればってところだね。[SEP]そうですね。ただ、そうですよね。
そうですね。おましをたくさんかけたり、夜は早めに部屋に入るようにしたり...[SEP]色々工夫してくれてるみたいだね。ありがとう。それが効果を発揮してくれればってところだね。[SEP]そうですね。ただ、大変ですよね。
そうですね。朝ましをたくさんかけたり、夜は早めにトイレに入るようにしたり...[SEP]色々工夫してくれてるみたいだね。ありがとう。それが効果を発揮してくれればってところだね。[SEP]そうですね。ただ、不思議ですよね。
ですね。まずはあれ(禁煙サポートプログラム)に申込んで許可をもらうところから始めてみようかなっと。[SEP]タバコをやめたいという意志があることが伺えます。しかし、タバコをやめることで体重が増加することに対する不安が大きいようですね。過去に禁煙に成功した経験から、食事の量が増えることを経験しているため、それが再び起こるかもしれないという心配があるようです。[SEP]
ですね。まずはあれ(禁煙サポートプログラム)に申込んでアドバイスをもらうところから始めてみようかなっと。[SEP]禁煙をやめたいという意志があることが伺えます。しかし、タバコをやめることで肥満が増加することに対する不安が大きいようですね。過去に禁煙に成功した経験から、食事の量が増えることを経験しているため、それが再び起こるかもしれないという心配があるようです。[SEP]
ですね。まずはあれ(禁煙サポートプログラム)に申込んでOKをもらうところから始めてみようかなっと。[SEP]たばこをやめたいという意志があることが伺えます。しかし、タバコをやめることで収入が増加することに対する不安が大きいようですね。過去に禁煙に成功した経験から、食事の量が増えることを経験しているため、それが再び起こるかもしれないという心配があるようです。[SEP]
そうですね。やっぱり、自分のために仕事は使うべきだという思いがあります。今の仕事は、それが嫌いなわけではないですが、自分が今やるべきことはそれではないと思います。自分の人生やキャリアの優先順位を上げるべきだと思います。[SEP]

そうですね。寝ましをたくさんかけたり、夜は早めに風呂に入るようにしたり...[SEP]そうですね、夜早く寝る努力や、ましをかける工夫、いいですね。ちょっとずつでも、そういう努力が結果につながると思いますよ。[SEP]そうですね。ただ、難しいですよね。
そうですね。昼ましをたくさんかけたり、夜は早めに部屋に入るようにしたり...[SEP]そうですね、夜早く寝る努力や寝ましをかける工夫、いいですね。ちょっとずつでも、そういう努力が結果につながると思いますよ。[SEP]そうですね。ただ、難しいですよね。
そうですね。おましをたくさんかけたり、夜は早めに家に入るようにしたり...[SEP]そうですね、夜早く寝る努力やおましをかける工夫、いいですね。ちょっとずつでも、そういう努力が結果につながると思いますよ。[SEP]そうですね。ただ、難しいですよね。
わからない。[SEP]仕事を続けたいという思いはあるものの、テレビでの使用機会がないため、やらなくてもいいのではという気持ちがあるということですね。役に立たないかもしれないという思いもある一方で、やった方がいいと感じる気持ちが少し大きいということですね。楽しく活動できる環境があれば、もっと前向きになれるかもしれません。[SEP]
わからない。[SEP]活動を続けたいという思いはあるものの、ステージでの使用機会がないため、やらなくてもいいのではという気持ちがあるということですね。役に立たないかもしれないという思いもある一方で、やった方がいいと感じる気持ちが少し大きいということですね。楽しく生活できる環境があれば、もっと前向きになれるかもしれません。[SEP]
わからない。[SEP]音楽を続けたいという思いはあるものの、ライブでの使用機会がないため、やらなくてもいいのではという気持ちがあるということですね。役に立たないかもしれないという思いもある一方で、やった方がいいと感じる気持ちが少し大きいということですね。楽しく練習できる環境があれば、もっと前向きになれるかもしれません。[SEP]
とても嬉しいです[SEP]OK,ではもしよければ,あなたが来られる前に一度気を取り直して,そのことについて話をしましょう。[SEP]そうしたいと思います,いいですね。
とても嬉しいです[SEP]OK,ではもしよければ,あなたが来られるように一度手を

そうですね。今は、応募作品の締め切りに追われている感じなので、もう少し余裕をもって準備をしたいです。[SEP]いいじゃないですか!早めに準備すると気分的にも余裕が持てそうですね。[SEP]そうなんです。今はいつもぎりぎりで、夜20時までに大きな郵便局まで荷物を出しに行くこともあるので、それが無いだけでも、だいぶ効率的になると思います。
そうですね。今は、応募受付の締め切りに追われている感じなので、もう少し余裕をもって準備をしたいです。[SEP]いいじゃないですか!早めに準備すると気分的にも余裕が持てそうですね。[SEP]そうなんです。今はいつもぎりぎりで、夜20時までに大きな郵便局まで顔を出しに行く必要もあるので、それが無いだけでも、だいぶ効率的になると思います。
そうですね。今は、応募者の締め切りに追われている感じなので、もう少し余裕をもって準備をしたいです。[SEP]いいじゃないですか!早めに準備すると気分的にも余裕が持てそうですね。[SEP]そうなんです。今はいつもぎりぎりで、夜20時までに大きな郵便局まで手紙を出しに行くところもあるので、それが無いだけでも、だいぶ効率的になると思います。
ええと,それって私と子どもの問題でしよ。私は煙草を吸ってますよ。う-ん,息子のそばでは吸わないように本当に気をつけています。でも妊娠していた時は除いて10年吸い続けています。シングルマザーでいるのはとてもストレスフルでまた,仕事もあります。だから,また吸いはじめてしまいました。[SEP]いろいろなことがあって,たばこを吸うことはリラックスする,ストレス解消法の一つになっているのですね。[SEP]そう,そうなんです。人によってはワインでしようし,私にはタバコなんです。
ええと,それって私と子どもの問題でしよ。私は煙草を吸ってますよ。う-ん,息子のそばでは吸わないように本当に気をつけています。でも妊娠していた時は除いて10年吸い続けています。シングルマザーでいることはとてもストレスフルでまた,仕事もあります。だから,また吸いはじめてしまいました。[SEP]いろいろな理由があって,たばこを吸うのはリラックスする,ストレス解消法の一つになっているのですね。[SEP]そう,そうなんです。人によってはワインでしようし,私にはタバコなんです。
ええと,それって私と子どもの問題でしよ。私は

お金を貯めたいと思うんですけどもたまらずに逆に失敗ばっかりです。[SEP]お金を貯めたいと思ってるんだけれど何かと、つい使いすぎてしまう。[SEP]使い過ぎたり、出費が多くてマイナスですね。
お金を貯めたいと思うんですけどもたまらずに逆に借金ばっかりです。[SEP]お金を貯めたいと思ってるんだけれど何かが、つい使いすぎている。[SEP]使い過ぎたり、出費が多くてマイナスですね。
お金を貯めたいと思うんですけどもたまらずに逆にわがままばっかりです。[SEP]お金を貯めたいと思ってるんだけれど何かを、つい使いすぎてます。[SEP]使い過ぎたり、出費が多くてマイナスですね。
はい。[SEP]OK,でも本当はどこかでたばこをやめたいと思っているように聞こえます。[SEP]ええ,そうすべきだと分かっています,毎年新年に“今年は禁煙する”といいますが,何かあるとできず,やめられないんです。何もなければやめられるのに・・・。
はい。[SEP]OK,でも本当はどこかでたばこをやめたいと思っているように聞こえます。[SEP]ええ,そうすべきだと分かっています,毎年新年に“今年は喫煙する”といいますが,何かあるとできない,やめられたんです。何もなければやめられるのに・・・。
はい。[SEP]OK,でも本当はどこかでたばこをやめたいと思っているように聞こえます。[SEP]ええ,そうすべきだと分かっています,毎年新年に“今年はそうする”といいますが,何かあるとできて,やめられたいんです。何もなければやめられるのに・・・。
いや、それが、特に何があったというわけではないんですが、間に合わなくて。[SEP]そっか、大事がなかったなら良かったよ。その通りに来ようとはしてくれたんだね。[SEP]そうですね。すみません。
いや、それが、特に何があったということではあるんですが、間に合わなくて。[SEP]そっか、大事がなかったなら良かったよ。予定通りに来ようとはしてくれたんだね。[SEP]そうですね。すみません。
いや、それが、特に何があったという意味では無いんですが、間に合わなくて。[SEP]そっか、大事がなかったなら良かったよ。思い通りに来ようとはしてくれたんだね。[SEP]そうですね。すみません。
もうないです。あの電コンを買うために電化製品家電屋さんの審査通したんですけど通らなかったんです。な

わからない。[SEP]前はカウンセリングの経験が役に立ち、周りの人と一緒にすることが楽しかったですが、現在は環境が変わり、参加することへのモチベーションが低下しているようです。それでも、以前の経験を思い出して再度挑戦しようと思うことがあり、今日もこうして参加しているということですね。[SEP]
わからない。[SEP]前はカウンセリングの方が役に立ち、他の人と一緒にすることが楽しかったですが、現在は環境が変わり、挑戦することへのモチベーションが低下しているようです。それでも、以前の経験を思い出して再度挑戦しようと思うことがあり、今日もこうして参加しているということですね。[SEP]
わからない。[SEP]前はカウンセリングの仕事が役に立ち、多くの人と一緒にすることが楽しかったですが、現在は環境が変わり、チャレンジすることへのモチベーションが低下しているようです。それでも、以前の経験を思い出して再度挑戦しようと思うことがあり、今日もこうして参加しているということですね。[SEP]
わかっています。わかっています。聞いたことがあるし,以前人に言われたこともあります。全部聞きました。ただどうやっていいかわからないんです。どうやって禁煙したらいいの?すごく難しい。[SEP]そうね,今までのような苦労をしなくてもあなたが利用できるものがあります。ニコチン代用品、パッチや,トローチ,ガム,吸入器,スプレー式点鼻薬など。チヤンテイックスやZyban(経口禁煙薬)にトライするのであれば,薬物治療について話をすることができます。禁煙グループへの参加もできますし,電話相談窓口もありますし...。[SEP]とにかくそれをする時間がない...
わかっています。わかっています。聞いた事があるし,以前人に言われた事もあります。全部聞きました。ただどうやっていいかわからないんです。どうやって禁煙したらいいの?すごく難しい。[SEP]そうね,今までのような苦労をしなくてもあなたが利用できるものがあります。ニコチン代用品、パッチや,トローチ,ガム,吸入器,スプレー式点鼻薬など。チヤンテイックスやZyban(経口禁煙薬)にトライするのであれば,薬物治療について話をする事ができます。禁煙グループへの参加もできますし,電話相談窓口もありますし...。[SEP]とにかくそれをする時間がない...
わかっていま

In [40]:
print(data_augmented.keys())
for i in range(10):
    print(data_augmented["train0"][i])

dict_keys(['train0', 'train1', 'train2', 'train3', 'train4'])
{'sentence': 'わ,わかりません。私が言いたいのは,息子の周りでは吸わないように気をつけているという事です。車の中では吸いませんし,息子が家にいるときは外に吸いに行きます。とにかく私にたばこが悪いというのは分かっているし,息子にとっても悪いタバコだと分かっているからタバコの周りにいないでほしいと言っているし,本当に気をつけています。[SEP]エイデンのためにもあなたの健康のためにもタバコをやめたほうがいいと思います。子どもの周りでタバコを吸うことが影響するのは耳の感染症だけではないのは知っていましたか?[SEP]うん', 'miti_code': 4, 'augmented': True, 'source': 'case3.csv'}
{'sentence': 'わ,わかりません。私が言いたいのは,息子の周りでは吸わないように気をつけているというのです。車の中では吸いませんし,息子が家にいるときは外に吸いに行きます。とにかく私にたばこが悪いという事は分かっているし,息子にとっても悪いものだと分かっているからタバコの周りにいないでほしいと言っているし,本当に気をつけています。[SEP]エイデンのためにもあなたの健康のためにもタバコをやめたほうがいいと思います。子どもの周りでタバコを吸うことが影響するのは耳の感染症だけではないのは知っていましたか?[SEP]うん', 'miti_code': 4, 'augmented': True, 'source': 'case3.csv'}
{'sentence': 'わ,わかりません。私が言いたいのは,息子の周りでは吸わないように気をつけているというものです。車の中では吸いませんし,息子が家にいるときは外に吸いに行きます。とにかく私にたばこが悪いという気持ちは分かっているし,息子にとっても悪いのだと分かっているからタバコの周りにいないでほしいと言っているし,本当に気をつけています。[SEP]エイデンのためにもあなたの健康のためにもタバコをやめたほうがいいと思います。子どもの周りでタバコを吸うことが影響するのは耳の感染症だけではないのは知っていましたか?[SEP]うん', 'miti_code':

In [41]:
p_list = ["p1/", "p2/", "p3/", "p4/", "p5/"]
dir_list = [base_path + "datasets/" + d for d in p_list]
filename_list = ["train.csv", "valid.csv", "test.csv"]
val_list = [valid_df1, valid_df2, valid_df3, valid_df4, valid_df5]
tes_list = [test_df1, test_df2, test_df3, test_df4, test_df5]

In [42]:
print(data_augmented.keys())

dict_keys(['train0', 'train1', 'train2', 'train3', 'train4'])


In [43]:
for i, key in enumerate(data_augmented.keys()):
    d = dir_list[i]
    if not os.path.exists(d):
        os.mkdir(d)
    else:
        shutil.rmtree(d)
        os.mkdir(d)
    train = pd.DataFrame.from_dict(data_augmented[key])
    valid = val_list[i]
    test = tes_list[i]
    train.to_csv(d+filename_list[0], index=False)
    valid.to_csv(d+filename_list[1], index=False)
    test.to_csv(d+filename_list[2], index=False)